<a href="https://colab.research.google.com/github/tyukei/LLMs-from-scratch/blob/feature%2Fch02/ch02/01_main-chapter-code/ch02_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
参考書 <a href="http://mng.bz/orYv">作りながら学ぶLLM入門</a> by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>参考コード: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>





# 第2章：テキストデータの操作

このノートブックで使用されているパッケージ：

In [1]:
! pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.4 MB/s eta 0:00:00


In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0+cu124
tiktoken version: 0.9.0


- この章では、LLMの入力データを準備としてデータの前処理とサンプリングを解説しています(下図)

```
LLMをつくりには、大きく以下の流れがあります。今回の前処理は、ゲームでいうSTAGE1-1にあたります。
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1単語の埋め込みを理解します

- このセクションにはコードによる説明はありません
- 埋め込みには多くの形態があります。(下図)この本のテキストの埋め込みに焦点を当てています
```
コンピュータは文字のまま扱えないため、一度数値に直してやります。
このように情報を数値化することを埋め込みと言います。
動画や音声もそれぞれの埋め込みモデルを使用し数値にすることができます。
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLMは、高次元空間（つまり、数千の次元）の埋め込みを使用します
- 人間は1、2、3次元まで考えても、このような高次元空間を視覚化できないため、2次元の埋め込み空間で簡易に理解します
```
似た単語は近いベクトルに集まっていることがわかります。
逆に関係ない単語は離れていることがわかります。
Googleなど検索エンジンもこのようにベクトルの近さで検索を行っています。
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2トークン化テキスト

- このセクションでは、テキストをトークン化します。これは、個々の単語や句読点などの小さなユニットにテキストを分割することを意味します

```
文章を単語に分けてみます。
言語によって分け方が異なりますが、一旦英語で考えてみます。
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 生のテキストデータをロードします
- [エディス・ウォートンによる評決](https://en.wikisource.org/wiki/The_Verdict)というパブリックドメインの短編小説を使用します

```
画家ジャック・ギズバーンが、名声の絶頂期に絵画を放棄し、富裕な未亡人と結婚してリヴィエラに移住した理由を探る物語
なぜこの小説が選ばれたのかは調査中。。。

```

In [3]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

-  `ssl.sslcertverificationError`が発生した場合、古いPythonバージョンを使用したことが原因の可能性があります。
https://github.com/rasbt/llms-from-scratch/pull/403
を参考に対応してください。

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目標は、LLM用にテキストをトークン化して埋め込むことです
- 読み込んだ上記の文章を使用し、いくつかの簡単なサンプルテキストに基づいて、簡単なトークンザーを開発しましょう
- 次の正規表現は、空白で分割します

```
re.split()
複数条件で文字列を分割する
```

In [5]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 空白だけでなくコンマやピリオドも分割したいので、それを行うために正規表現を変更しましょう

In [6]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 空の文字列が作成されていますので、それらを削除しましょう
```
strip()
両端にある特定の文字列や空白文字を削除するメソッド
```

In [7]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- これでも良さそうですが、！や？など、他の種類の文字も処理しましょう

In [8]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- これで完了です。このトークン化を利用し、生のテキストデータを処理する準備ができました

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- トークンの総数を計算しましょう

```
ダブりあり
```

In [10]:
print(len(preprocessed))

4690


## 2.3トークンをトークンIDに変換します

- 次に、埋め込み処理できるようテキスをトークンIDに変換します。

```
単語をキー、IDを値とした辞書を作ります。
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 下記トークンから、すべてユニークなトークンで構成される語彙を構築できるようになりました

```
set()
重複を除くlist

sorted()
特定のルール(アルファベット順、数値順、文字数順)で順番を並び替える
```

In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


```
辞書を作成している。
単語(token)をキー、トークンID(integer)を値

enumerate()
リストやタプルなどの反復可能なオブジェクトの要素に、連番（インデックス）を付与して取得できる

```

In [12]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 以下は、この語彙の最初の50エントリです。
```
items()
辞書の中のすべてのキーと値の組み合わせの一覧を取得
```

In [13]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 以下では、小さな辞書を使用して短いサンプルテキストのトークン化を示します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- これまでの実装をトークナイザークラスに集約します

```
これまでやってきたこと
- 文章をスペースなど区切りで単語に分ける
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

- 空文字の除去
item.strip() for item in preprocessed if item.strip()

- (new)辞書を使用し、単語 <=> トークンIDを取得
str_to_int(vocab): {単語: トークンID}という辞書(ex. apple:1)
int_to_str: {トークンID: 単語}という辞書(ex. 1:apple)
{i:s for s,i in vocab.items()}でキーと値を入れ替えている

preprocessed...textの単語に分けたlist(['apple','banana'])
s...preprocessedの要素の1つ(apple)
self.str_to_int[s]... sをキーとした値(トークンID)の取得

self.int_to_str[i]...i(トークンID)をキーとした値(単語)の取得
```

In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- 「エンコード」関数はテキストをトークンIDに変えます
- 「デコード」関数はトークンIDをテキストに戻します

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- トークンザーを使用してテキストを整数にエンコードすることができます（つまり、トークン化）
- これらの整数は、LLMの入力として（後で）埋め込むことができます

In [15]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 整数をテキストにデコードすることができます

In [16]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

```
encodeとdecodeで同じ結果が返ってくることが確認できます
```

In [17]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4特別なコンテキストトークンの追加

- 不明な単語のために「特別な」トークンを追加し、テキストの終わりを示すことは便利です

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一部のトークンザーは、追加のコンテキストでLLMを支援するために特別なトークンを使用しています
-  `[BOS]`（Begin Of Sequence）テキストの始まりをマークします
-  `[EOS]`（End Of Sequence）テキストが終了するマークします。（これは通常、複数の無関係なテキストを連結させるのにつかいます。例えば2つの異なるウィキペディア記事または2つの異なる本など）
-  `[PAD]`（Padding）1を超えるバッチサイズのLLMをトレーニングする場合にしようします。(例えば、長さを揃えるために、異なる長さの複数のテキストデータに使用するなど)
-  `[UNK]`語彙に含まれていない単語を表す
- GPT-2は上記のこれらのトークンを必要としないが、 `<| endoftext |>`トークンのみ使用して複雑さを減らしています。
-` <| endoftext |> `は上記の` [EOS] `トークンに類似しています
-GPTはパディングに `<| endoftext |>`を使用します（通常、バッチ入力でトレーニングするときにマスクを使用するため）
-GPT-2は、表記の単語に`UNK`トークンを使用しませが、代わりに、Gバイトペアエンコード（BPE）トークンザーを使用します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 次のテキストをトークン化するとどうなるか見てみましょう。

In [18]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上記は、語彙に「hello」という単語が含まれていないため、エラーを生成します
- そのような場合に対処するために、` <| unk |>`のような特別なトークンを語彙に追加して、未知の単語を表すことができます
- すでに語彙を拡張しているので、「 "<| endoftext |>" `という別のトークンを追加しましょう。これは、テキストの終わりを示すためにGPT-2トレーニングで使用されます（トレーニングデータセットが複数の記事、本などで構成されている場合など、連結テキスト間でも使用されます。

In [19]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

```
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)
で実行したときより+2されていることがわかります
```

In [20]:
len(vocab.items())

1132

In [21]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- それに応じて、新しい `unk`トークンをいつ、どのように使用するかを知るために、トークナイザーを調整する必要があります

```
item if item in self.str_to_int
もし、文字列の中に単語があれば、その単語を
else "<|unk|>" for item in preprocessed
なければ、unkを
返す
```

In [22]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

修正されたトークン剤でテキストをトークン化してみましょう。

In [23]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [24]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [25]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5バイトペアエンコーディング

- GPT-2は、トークナイザーとしてbytepairエンコード（BPE）を使用しました
- より小さなサブワード単位か場合によってはばらばらの文字に分化します。
- たとえば、GPT-2の語彙に`unfamiliarword`という単語がない場合、訓練されたBPEの合併に応じて、["unfam", "iliar", "word"] またはその他のサブワードの内訳としてそれをトークン化するかもしれません。
- 元のBPEトークナイザーはこちらをご覧ください：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- この章では、Openaiのオープンソース[Tiktoken](https://github.com/openai/tiktoken)ライブラリのBPEトークナイザーを使用しています。
-[./bytepair_encoder](../02_bonus_bytepair-encoder) にノートブックを作成しました.これらの2つの実装を並べて比較しました（サンプルテキストではTiktokenは約5倍高速でした）

```
BPEは複雑になるので、OSSのtiktokenを使います。
GPT-2, GPT-3, ChatGPTで使われていることからわかるように、OpenAIが作ったものです。
```

In [26]:
! pip install tiktoken

In [27]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [28]:
tokenizer = tiktoken.get_encoding("gpt2")

```
トークナイザの使い方は実装したSimpleTokenizerV2のクラスencode()と同じです。
```

```
トークンIDをみてみると、50256という大きい値があるが、
ChatGPTのBPEのトークナイザのサイズは50257であり、
最後のトークンIDは、<|endoftext|>であることを示しています。
```

In [29]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


```
トークナイザの使い方は実装したSimpleTokenizerV2のクラスdecode()と同じです。
```

In [30]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPEトークナーザーは、未知の単語をサブワードと個々の文字に分割します。

```
どの粒度に切り分けられるかは、
多くの単語に共通するサブワードに切り分けられます。
例えば、"define"という言葉の場合、
d,e,f,i,n,eと個々の文字に分け、
一緒に出現する頻出が高い文字の組み合わせに分けます。
deは、"depend","made","hidden"など共通するため"de"にサブワードにマージされる可能性がある。
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6スライドウィンドウを使用したデータサンプリング

- LLMを一度に1語ずつ生成するようにトレーニングするため、次の単語がシーケンスの次の単語が予測するターゲットを表している場合に、トレーニングデータを準備する必要があります。

```
LLMの訓練に限らず、機械学習の訓練は、入力と出力(入力変数と目的変数)のペアを用意し、誤差を最小にするよう最適化を行ないます。
LLMの場合、入力が１単語でなく複数の単語になります。
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [31]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 各テキストチャンクについて、入力とターゲットが必要です
- モデルに次の単語を予測することを望むので、ターゲットは入力が右に1つの位置でシフトされたものです

In [32]:
enc_sample = enc_text[50:]

```
context_sizeは、入力に含まれるトークン数
```

In [33]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


-  1つずつ、予測は次のようになります。

In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [35]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- attention機構解説の時、次の単語の予測を処理します
- 今のところ、入力データセットを反復し、1つによってシフトされたターゲットを返すシンプルなデータローダーを実装します

- Pytorchのインストールとインポート（インストールのヒントについては、付録Aを参照）

In [36]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu124


- 位置を+1するスライドウィンドウアプローチを使用します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 入力テキストデータセットからチャンクを抽出するデータセットとデータローダーを作成する

```
PyTorchのDatasetクラスを継承している

input_ids
入力変数

target_ids
目的変数

token_ids
テキスト全体をトークン化

--len--()
データセットに含まれている行の総数

__getitem__()
データセットから１行返す
```

In [37]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

```
GPTDatasetV1とPyTorchのDataLoderを使って入力をバッチで読み込むコード

GPTDatasetV1は、1つのバッチを設定している。

DataLoaderのパラメータにbatch_sizeがあるように、バッチが設定できる。
バッチにする理由は、計算処理の速度を速めるためと思われる(並列化)

```

In [38]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- コンテキストサイズが4のLLMに対して、バッチサイズが1のバッチサイズでDataloaderをテストしましょう。

In [39]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [40]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [41]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- コンテキストの長さ（例:4）に等しいストライドを使用する例は以下のとおりです

```
Q なんで、目的変数もサイズは４？今までの流れだと1では？
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">


- バッチ出力を作成することもできます
- オーバラップは過学習を起こすので、オーバラップをさせないために、スライドを使用します

```
バッチ処理もできるか確認。
```

In [42]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7トークンの埋め込みを作成します

- ここまででデータの準備はぼほできています。
- しかし最後に、埋め込み層を使用してトークンを連続ベクトル表現に埋め込みましょう
- 通常、これらの埋め込み層はLLM自体の一部であり、モデルトレーニング中に更新（トレーニング）されます

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- 入力IDとして 2、3、5、および1の次の4つの入力例があるとします（トークン化後）：

In [56]:
input_ids = torch.tensor([2, 3, 5, 1])

- 簡単にするために、6語の小さな語彙で、サイズ3次元の埋め込みを作成したいとします。

```

torch.manual_seed(123)
では、ランダムシード123に設定し、ランダムなテンソルを作っている
初回は適当に数値を作って、学習の過程でより良い値にする感じ
```

In [58]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- これにより、6x3の多重行列が作成されます。

In [59]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


-  上記のレイヤーアプローチは、本質的に1ホットエンコードを実装するより効率的な方法。
- 埋め込み層は、ワンホットのエンコーディングおよび行列拡大アプローチに相当するより効率的な実装であるため、逆伝播法を介して最適化できるニューラルネットワーク層と見なすことができます。
```
ワンホットエンコーディングに詳しい方は読んでねぐらい
正直わかってない。。。
```

-  ID 3のトークンを3次元ベクトルに変換するには、次のことを行います。
```
さっき適当に入力IDとして 2、3、5、1としたものの一つ。
```

In [46]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 上記は `embedding_layer`多重マトリックスの4列目であることに注意してください
- 上記の4つの `input_ids`値すべてを埋め込むには、以下のようにします。

```
さっき出したやつの４列目
pythonはlistを0から数えるため、
要素0とは1列目
要素1とは2列目
要素2とは3列目
要素3とは4列目
...

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
```

```
全トークンID入れてみる
input_ids = torch.tensor([2, 3, 5, 1])
```

In [47]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 埋め込み層は基本的にルックアップ操作です。
```
ルックアップ演算
用語	意味
look-up operation	インデックスやキーでデータを取得する操作
行列での例	行・列・要素を位置指定で取得（インデックス参照）
NLPでの例	単語IDで埋め込みベクトルを取得


埋め込みベクトルの[トークンID+1]行をとって並べる
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

-  **埋め込み層を通常の線形層と比較するボーナスコンテンツに興味があるかもしれません：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8ワードポジションのエンコード

- 埋め込み層IDを入力シーケンスのどこにあるかに関係なく、IDを同一のベクトル表現に変換しています。

```
例えば、foxという単語が1個目、4個目と出てきた場合
現在はどちらも同じ埋め込みベクトルになっている
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置埋め込みは、トークン埋め込みベクトルと組み合わされて、大規模な言語モデルの入力埋め込みとして使用されます。

```
位置情報は大事。。。

私が好きなのは、カレー
カレーが好きなのは、私
```

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- Bytepairエンコーダーの語彙サイズは50,257です。
- 入力トークンを256次元ベクトル表現にエンコードするとします。

```
GPT3は、12288次元
```

In [60]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- Dataloaderのデータをサンプリングすると、各バッチのトークンを256次元ベクトルに埋め込みました
- それぞれ4トークンの8つのバッチサイズがある場合、これにより8 x 4 x 256テンソルになります。

In [49]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

```
まずはトークンIDを出してみる
```

In [61]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


```
埋め込みベクトルに直してみる
```

In [62]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2は絶対位置埋め込みを使用するため、別の埋め込み層を作成します。

```
位置埋め込みには相対と絶対があります。
相対：あるトークンとトークンとの距離
絶対: ある位置を０としてそこからの距離

torchにある関数を使用する
pos_embedding_layer
イメージは
バッチ１の１トークン目、２トークン目、３トークン目、４トークン目は、
1.1, 1.2, 1.3, 1.4...
バッチ2の１トークン目、２トークン目、３トークン目、４トークン目は、
2.1, 2.2, 2.3, 2.4...
...
```


In [65]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[-0.2899,  0.7224,  0.3544,  ...,  0.8822,  0.1659, -0.6570],
        [-0.9077,  1.1814, -2.1400,  ...,  0.2671, -1.6955, -1.1446],
        [-0.7482, -1.0564,  0.2795,  ..., -0.3304,  1.6200,  0.4960],
        [-0.4959, -1.1469, -0.4270,  ...,  0.8587, -0.8780, -0.4342]],
       requires_grad=True)


```
位置エンべディングの値をこれまでの埋め込みベクトルに足し合わせる

New埋め込み = old埋め込み+位置埋め込み
```

In [66]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


-  LLMで使用される入力埋め込みを作成するには、トークンと位置埋め込みを追加するだけです。

In [69]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


- 入力処理ワークフローの初期段階では、入力テキストが個別のトークンにセグメント化されています
- このセグメンテーションに続いて、これらのトークンは、事前定義された語彙に基づいてトークンIDに変換されます。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

＃概要とテイクアウト

[./Dataloader.ipynb

エクササイズソリューションについては、[./exercise-solutions.ipynb]（./exercise-solutions.ipynb）を参照してください。

[バイトペアエンコーディング（BPE）トークナイザーをゼロから参照]